## 구글 드라이브 마운트
colab 런타임을 본인의 구글 드라이브에 마운트시킨 후, 적절한 경로를 이동해 줍니다.

In [1]:
import os
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

# morp 폴더로 경로 이동
path = '/content/drive/MyDrive/cakd3_colab/2nd_project/bert_classification_kor/morp' 
os.chdir(path)

Mounted at /content/drive


## 환경 설정
KorBERT를 활용할 수 있는 환경을 설정해 줍니다. 우선 아래와 같은 모듈들을 설치해 줍니다.
  - tensorflow-gpu==1.15
  - tensorflow-hub==0.7.0


그 후, 사용 가능한 gpu를 환경변수 CUDA_VISIBLE_DEVICES 변수에 설정해 줍니다.

In [2]:
!pip install tensorflow-gpu==1.15  # 버전 충돌로 다른 버전으로 설치
!pip install tensorflow-hub==0.7.0

     |████████████████████████████████| 411.5 MB 7.3 kB/s 
     |████████████████████████████████| 3.8 MB 75.4 MB/s 
     |████████████████████████████████| 503 kB 76.2 MB/s 
     |████████████████████████████████| 50 kB 8.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=7f8e26b0b3f2acf9e8348ef8592e7829c3225020afd834d566eff15fe0ded4d7
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succe

사용 가능한 gpu를 확인해 줍니다. 만일 런타임 유형 변경에서 가속기로 gpu를 설정하지 않는다면 다음과 같은 에러 메시지가 나올 것입니다.

`NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.`

이럴 경우 런타임 유형을 변경한 후 다시 실행합니다.

In [3]:
!export CUDA_VISIBLE_DEVICES=0 # 0번 gpu를 쓰겠다
!nvidia-smi # 현재 GPU 사용하고 있는 현황이 나옴

Mon Nov  1 08:43:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# checkpoint -> keras layer
ETRI의 KorBERT를 keras model로 변환해 줍니다. 이는 `bert_to_keras.py`로 가능합니다.

In [ ]:
# bert_module로 변환해 줬으면 이 과정은 생략하셔도 됩니다.
# kobert -> 케라스 레이어로 변환
!python bert_to_keras.py 

From bert_to_keras.py:29: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

From /content/drive/My Drive/cakd3_colab/2nd_project/bert_classification_kor/morp/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

0.02 ---------
From /content/drive/My Drive/cakd3_colab/2nd_project/bert_classification_kor/morp/modeling.py:194: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

From /content/drive/My Drive/cakd3_colab/2nd_project/bert_classification_kor/morp/modeling.py:435: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

From /usr/local/lib/python3.7/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint 

In [ ]:
os.listdir()  
# 앞에 bert_to_keras.py를 실행하면 bert-module 폴더가 생성됨(eojeol의 경우)
# morp의 경우에는 'bert-module_tmp'이 생성되어 구글 드라이브에서 bert-module로 폴더명 수정함(추후 코드 수정 필요) 

['eval_bert_finetuning.py',
 'utils.py',
 'inference.py',
 'bert_to_keras.py',
 'train.log',
 'modeling.py',
 'README',
 'train_bert_finetuning.py',
 'prepare_data.py',
 '.DS_Store',
 '002_bert_morp_tensorflow',
 'layers',
 'vectorizers',
 'models',
 'readers',
 '__pycache__',
 'bert-module_tmp']

## 데이터 전처리
KorBERT가 Fine-Tune을 할 수 있게끔 데이터를 준비해 주어야 합니다. 준비해야 하는 데이터 파일은 tab(\t)을 기준으로 label과 input sequence가 나뉘어 있어야 합니다.

`prepare_data.py`를 실행할 경우 data -> label, seq.in 으로 나뉘게 되는데, 이떄 seq.in은 KorBERT가 pre-train된 대로 tokenization 과정을 거치게 됩니다.

`prepare_data.py`의 경우 mandatory한 인자로 --input(-i)와 --output(-o)를 갖습니다.

`python prepare_data.py --input ${input_data} --output ${processed_data_path}`


In [ ]:
# !python prepare_data.py --input starbucks_data/train/train_data.txt --output starbucks_data/train/
# sample_data.txt 파일(단톡방에 공유받은 파일)을 input으로 넣고, label과 seq in 파일이 sample_data라는 폴더에 output으로 저장됨

"""
prepare_data.py

options:
--input  [-i] [필수] txt File        # label과 text가 \t로 구분된 txt파일
--output [-o] [필수] Path to data    # 변환된 파일을 저장할 경로
"""

!python prepare_data.py --input train.txt --output train_data  
!python prepare_data.py --input validation.txt --output validation_data  
!python prepare_data.py --input test_revise.txt --output test_data  


In [5]:
from vectorizers import tokenizationK as tk


sample_sentence = '와 이 커피 진짜 맛있다.'
### 소스코드를 참조하여 알맞게 채워보세요 ###
# vocab_file_path를 vocab.korean.rawtext.list파일 경로를 지정해주어야 토큰화 작업이 가능해진다.
vocab_file_path = '/content/drive/MyDrive/cakd3_colab/2nd_project/bert_classification_kor/morp/002_bert_morp_tensorflow/vocab.korean_morp.list'

tokenizer = tk.FullTokenizer(vocab_file_path)

tokens = "" # 원래 처음 토큰은 "" 아무것도 없음

for word in sample_sentence.split():
    word_tokens = " ".join(tokenizer.tokenize(word)) # 토크나이저를 통해 샘플 문장에서 공백으로 분리된 각 어절이 " "로 이어진다.  
    tokens += word_tokens + " "

In [6]:
sample_sentence.split()

['와', '이', '커피', '진짜', '맛있다.']

In [7]:
tokens # 토큰화 결과

'와  이  커피  진짜  맛있 다 .  '

## Fine-Tuning 훈련

준비된 데이터를 바탕으로 KorBERT fine-tuning을 실시해 줍니다.

`train_bert_finetuning.py`를 실행하며, mandatory 인자로는 --train(-t) 와 --save(-s)를 받습니다.

`python train_bert_finetuning.py --train ${processed_data_path} --save ${saved_model}`

In [ ]:
""" 
train_bert_finetuning.py    # 거의 model.fit이라고 생각하면 됨

option:
--train     [-t]    [필수]              Path to training data                   # training_data가 있는 폴더 경로
--val       [-v]    [default = False]   Path to validation data                 # validation_data가 있는 폴더 경로
--save      [-s]    [필수]              Folder path to save the trained model   # model을 저장할 폴더 경로
--epochs    [-e]    [defalut = 5]       Numbers of epochs                       
--batch     [-bs]   [default = 64]      Batch size                              
--type      [-tp]   [default = bert]    'bert' or 'albert'                      # bert나 albert 중 어떤것을 사용할 것인지

"""
!python train_bert_finetuning.py --train train_data --val validation_data --save train_model  


2021-11-01 07:47:33.245661: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-01 07:47:33.251032: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-11-01 07:47:33.251478: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a6d5d20a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-01 07:47:33.251534: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-01 07:47:33.253533: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-01 07:47:33.469692: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-01 07:47:33.470985:

## EVALUATION

In [ ]:
# tensorflow의 버전이 맞지않아 decode 오류 발생 -> 추가적인 모듈 다운로딩 필요
!pip install h5py==2.10.0  # 버전 충돌로 h5py 라이브러리를 새로 설치 

     |████████████████████████████████| 2.9 MB 4.2 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires gast==0.4.0, but you have gast 0.2.2 which is incompatible.
tensorflow 2.6.0 requires h5py~=3.1.0, but you have h5py 2.10.0 which is incompatible.
tensorflow 2.6.0 requires tensorboard~=2.6, but you have tensorboard 1.15.0 which is incompatible.
tensorflow 2.6.0 requires tensorflow-estimator~=2.6, but you have tensorflow-estimator 1.15.1 which is incompatible.


In [ ]:
# fine_tuning으로 생성한 output 모델을 input으로 넣고 test data로 평가 진행

"""
eval_bert_finetuning.py  # model.evaluate 생각하면 될 듯

options:

--model [-m]    [필수]              Path to Bert            # 위의 train_bert_finetuning.py 에서 생성한 모델폴더
--data  [-d]    [필수]              Path to data            # test_data가 들어있는 폴더
--type  [-tp]   [defalut = 'bert']  'bert' or 'albert'      

"""
!python eval_bert_finetuning.py --model train_model --data test_data 



2021-11-01 08:03:13.730917: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-01 08:03:13.736149: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-11-01 08:03:13.736670: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x560167e40a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-01 08:03:13.736726: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-01 08:03:13.738684: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-01 08:03:13.742613: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-01 08:03:13.742675: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:16

In [ ]:
with open('/content/drive/MyDrive/cakd3_colab/2nd_project/bert_classification_kor/morp/train_model/test_results/test_total.txt', 'r') as file:
  result = file.read()
print(result)

Intent accuracy = 0.8505457598656591

